In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
batch_size = 100
num_classes = 10
epochs = 150
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#### LOAD AND TRANSFORM
# The output of torchvision datasets are PILImage images of range [0, 1].
# https://pytorch.org/docs/stable/torchvision/transforms.html
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomAffine(degrees=20, translate=(0.2, 0.2), scale=(1.0, 1.2)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [4]:
test_transforms = torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                  ])

testset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [5]:
x_batch, y_batch = iter(train_loader).next()
print("Training set: {} samples - Max value: {} - Min value: {}".format(len(train_loader.dataset), 
                                                                        x_batch.max(), x_batch.min()))

Training set: 50000 samples - Max value: 2.7537312507629395 - Min value: -2.429065704345703


In [6]:
x_batch, y_batch = iter(test_loader).next()
print("Test set: {} samples - Max value: {} - Min value: {}".format(len(test_loader.dataset), 
                                                                        x_batch.max(), x_batch.min()))

Test set: 10000 samples - Max value: 2.7537312507629395 - Min value: -2.429065704345703


In [7]:
print("Example batch shape: {}".format(x_batch.shape))

Example batch shape: torch.Size([100, 3, 32, 32])


In [8]:
class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """

    def __init__(self, sigma=0.1, is_relative_detach=True):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.noise = torch.tensor(0).to(device).float()

    def forward(self, x):
        if self.training and self.sigma != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.repeat(*x.size()).normal_() * scale
            x = x + sampled_noise
        return x 

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block1 = self._CBGN(3, 32)
        self.block2 = self._CBGN(32, 64)
        self.block3 = self._CBGN(64, 128)
        self.block4 = self._CBGN(128, 256)
        self.block5 = self._CBGN(256, 512)
        # Flatten at forward!
        self.features = nn.Linear(512, 512)
        self.reluFeatures = nn.ReLU()
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.block1(x)               # 32x32 -> 16x16 @ 32
        out = self.block2(out)             # 16x16 -> 8x8 @ 64
        out = self.block3(out)             # 8x8 -> 4x4 @ 128
        out = self.block4(out)             # 4x4 -> 2x2 @ 256
        out = self.block5(out)             # 2x2 -> 1x1 @ 512
        out = out.view(out.size(0), -1)    # Flatten
        out = self.reluFeatures(self.features(out))  # 512 -> 512
        out = self.classifier(out)         # 512 -> num_classes (10)
        return out

    # DEF A BLOCK Conv + BN + GN + MaxPool
    def _CBGN(self, in_channels, filters):
        layers = []
        
        layers += [nn.Conv2d(in_channels, filters, kernel_size=(3,3), padding=1)]
        layers += [nn.BatchNorm2d(filters)]
        layers += [GaussianNoise(0.3)]
        layers += [nn.ReLU()]
        
        layers += [nn.Conv2d(filters, filters, kernel_size=(3,3), padding=1)]
        layers += [nn.BatchNorm2d(filters)]
        layers += [GaussianNoise(0.3)]
        layers += [nn.ReLU()]
        
        layers += [nn.MaxPool2d(kernel_size=(2,2))]
        return nn.Sequential(*layers)


net = Net().to(device)
print(net)

Net(
  (block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GaussianNoise()
    (3): ReLU()
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GaussianNoise()
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GaussianNoise()
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GaussianNoise()
    (7): ReLU()
    (8): MaxPool2d(kernel_size

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, weight_decay=1e-6)

In [11]:
# lr = 0.1     if epoch < 50
# lr = 0.01    if 30 <= epoch < 100
# lr = 0.001   if epoch >= 100
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100], gamma=0.1)

In [12]:
print("\n---- Start Training ----")
best_accuracy = -1
for epoch in range(epochs):

    # TRAIN THE NETWORK
    train_loss, train_correct = 0, 0
    net.train()
    for inputs, targets in train_loader:
        # data is a list of [inputs, labels]
        inputs, targets = inputs.to(device), targets.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        _, pred = outputs.max(1)  # get the index of the max log-probability
        train_correct += pred.eq(targets).sum().item()

        # print statistics
        train_loss += loss.item()
        
    train_loss /= len(train_loader.dataset)

    # TEST NETWORK
    net.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            test_loss += criterion(outputs, targets)
            _, pred = outputs.max(1)  # get the index of the max log-probability
            correct += pred.eq(targets).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    for param_group in optimizer.param_groups:
        current_lr = param_group['lr']
    print("[Epoch {}] LR: {:.3f} - Train Loss: {:.5f} - Test Loss: {:.5f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(epoch+1, current_lr, train_loss, test_loss, 100. * train_correct / len(train_loader.dataset), test_accuracy))
    
    if test_accuracy>best_accuracy:
        best_accuracy = test_accuracy
    
    scheduler.step()
    
print('Finished Training')
print("Best Test accuracy: {:.2f}".format(best_accuracy))


---- Start Training ----
[Epoch 1] LR: 0.100 - Train Loss: 0.01692 - Test Loss: 0.01488 - Train Accuracy: 36.74% - Test Accuracy: 45.85%
[Epoch 2] LR: 0.100 - Train Loss: 0.01335 - Test Loss: 0.01198 - Train Accuracy: 51.36% - Test Accuracy: 56.35%
[Epoch 3] LR: 0.100 - Train Loss: 0.01175 - Test Loss: 0.01051 - Train Accuracy: 57.78% - Test Accuracy: 61.83%
[Epoch 4] LR: 0.100 - Train Loss: 0.01074 - Test Loss: 0.00964 - Train Accuracy: 61.79% - Test Accuracy: 65.12%
[Epoch 5] LR: 0.100 - Train Loss: 0.00990 - Test Loss: 0.00897 - Train Accuracy: 64.92% - Test Accuracy: 67.01%
[Epoch 6] LR: 0.100 - Train Loss: 0.00932 - Test Loss: 0.00770 - Train Accuracy: 67.29% - Test Accuracy: 72.20%
[Epoch 7] LR: 0.100 - Train Loss: 0.00885 - Test Loss: 0.00782 - Train Accuracy: 68.77% - Test Accuracy: 72.05%
[Epoch 8] LR: 0.100 - Train Loss: 0.00846 - Test Loss: 0.00731 - Train Accuracy: 70.26% - Test Accuracy: 73.73%
[Epoch 9] LR: 0.100 - Train Loss: 0.00813 - Test Loss: 0.00722 - Train Accurac

[Epoch 74] LR: 0.010 - Train Loss: 0.00348 - Test Loss: 0.00353 - Train Accuracy: 87.69% - Test Accuracy: 88.04%
[Epoch 75] LR: 0.010 - Train Loss: 0.00349 - Test Loss: 0.00348 - Train Accuracy: 87.65% - Test Accuracy: 88.12%
[Epoch 76] LR: 0.010 - Train Loss: 0.00346 - Test Loss: 0.00352 - Train Accuracy: 87.72% - Test Accuracy: 87.90%
[Epoch 77] LR: 0.010 - Train Loss: 0.00347 - Test Loss: 0.00352 - Train Accuracy: 87.75% - Test Accuracy: 88.01%
[Epoch 78] LR: 0.010 - Train Loss: 0.00344 - Test Loss: 0.00356 - Train Accuracy: 87.87% - Test Accuracy: 87.68%
[Epoch 79] LR: 0.010 - Train Loss: 0.00343 - Test Loss: 0.00353 - Train Accuracy: 87.81% - Test Accuracy: 87.95%
[Epoch 80] LR: 0.010 - Train Loss: 0.00343 - Test Loss: 0.00354 - Train Accuracy: 87.80% - Test Accuracy: 87.89%
[Epoch 81] LR: 0.010 - Train Loss: 0.00344 - Test Loss: 0.00352 - Train Accuracy: 87.80% - Test Accuracy: 88.07%
[Epoch 82] LR: 0.010 - Train Loss: 0.00342 - Test Loss: 0.00351 - Train Accuracy: 87.90% - Test 

[Epoch 147] LR: 0.001 - Train Loss: 0.00319 - Test Loss: 0.00348 - Train Accuracy: 88.65% - Test Accuracy: 88.17%
[Epoch 148] LR: 0.001 - Train Loss: 0.00318 - Test Loss: 0.00347 - Train Accuracy: 88.73% - Test Accuracy: 88.20%
[Epoch 149] LR: 0.001 - Train Loss: 0.00321 - Test Loss: 0.00345 - Train Accuracy: 88.51% - Test Accuracy: 88.30%
[Epoch 150] LR: 0.001 - Train Loss: 0.00323 - Test Loss: 0.00346 - Train Accuracy: 88.50% - Test Accuracy: 88.13%
Finished Training
Best Test accuracy: 88.30
